In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 1. 9개 변수 리스트 (컬럼명을 본인의 파일에 맞게 수정하세요)
cols_9 = [
    'delta_bok',    # 금리 변동
    'cpi_infl',     # 물가
    'output_gap',   # 산출갭(HP필터)
    'tone_mkt',     # 우리 어조 지수 (핵심)
    'tone_google',  # 구글 번역 지수
    'tone_basic',   # 일반 형태소 지수
    'media_idx',    # 미디어 지수
    'uncertainty',  # 불확실성 지수
    'call_rate_lag' # 과거 금리 등
]

# 2. 상관계수 계산
corr_9 = df_final[cols_9].corr()

# 3. 히트맵 시각화
plt.figure(figsize=(12, 10))
sns.heatmap(corr_9, 
            annot=True, 
            fmt=".2f", 
            cmap='coolwarm', # 논문 느낌 나는 색상
            linewidths=0.5)

plt.title('Correlation Matrix: Comprehensive Analysis (9 Variables)')
plt.show()

In [ ]:
import statsmodels.api as sm
import pandas as pd

import pandas as pd
import statsmodels.api as sm

def compare_ols_methods(df, target_col, feature_cols):
    """
    White(Robust)와 HAC(Newey-West) 보정 방식을 비교하는 함수
    """
    # 1. 독립변수(X)와 종속변수(y) 설정
    X = sm.add_constant(df[feature_cols])
    y = df[target_col]
    
    # 2. White (Heteroskedasticity-Robust) 모델 - HC1 옵션
    # 데이터에 HP필터 적용으로 어느정도 자기상관 정제된 데이터임을 가정
    model_white = sm.OLS(y, X).fit(cov_type='HC1')
    
    # 3. HAC (Newey-West) 모델 - 시계열 자기상관 보정(일반적으로 std error가 white의 경우보다 크게 계산됨)
    # maxlags는 보통 데이터 특성에 따라 설정하며, 여기서는 4(분기 수준)로 설정
    model_hac = sm.OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags': 4})
    
    # 4. 비교 결과 취합
    results = []
    for col in X.columns:
        results.append({
            'Variable': col,
            'Coef': model_white.params[col],
            'White_Std.Err': model_white.bse[col],
            'White_P-value': model_white.pvalues[col],
            'HAC_Std.Err': model_hac.bse[col],
            'HAC_P-value': model_hac.pvalues[col]
        })
    
    # 데이터프레임으로 변환
    comparison_df = pd.DataFrame(results)
    
    print(f"=== OLS 보정법 비교 결과 (R-squared: {model_white.rsquared:.4f}) ===")
    return comparison_df.round(4)

# --- 사용 예시 ---
# feature_cols = ['cpi_infl', 'output_gap', 'tone_mkt']
# target_col = 'delta_bok'
# compare_ols_methods(df_final, target_col, feature_cols)


In [ ]:
# ---------------------------------------------------------
# 4.1 & 4.2. 테일러 준칙 회귀 모델 구축 및 OLS 추정
# ---------------------------------------------------------

# [모델 1] Baseline: 거시경제 변수만 사용 (논문 목표 R2 = 0.08)
# DMP = alpha + beta1*IP_gap + beta2*CPI
model_baseline = run_robust_ols('delta_bok', ['output_gap', 'cpi_infl'], df_final)

# [모델 2] Augmented (우리 모델): 거시경제 + tone_mkt 추가 (논문 목표 R2 = 0.37)
# DMP = alpha + beta1*IP_gap + beta2*CPI + gamma*tone_mkt
model_mkt = run_robust_ols('delta_bok', ['output_gap', 'cpi_infl', 'tone_mkt'], df_final)

# ---------------------------------------------------------
# 4.3. 타 지표와의 성능 비교 (Robustness Check)
# ---------------------------------------------------------

# [모델 3] 구글 번역본(영문 분석) 기반
model_google = run_robust_ols('delta_bok', ['output_gap', 'cpi_infl', 'tone_google'], df_final)

# [모델 4] 일반 형태소 분석기 기반
model_basic = run_robust_ols('delta_bok', ['output_gap', 'cpi_infl', 'tone_basic'], df_final)

# ---------------------------------------------------------
# 결과 출력 및 비교 리포트
# ---------------------------------------------------------

print("=== [4.2] 메인 회귀분석 결과 (Augmented Taylor Rule) ===")
print(model_mkt.summary())

print("\n=== [4.3] 모델별 성능(R-squared) 비교 결과 ===")
summary_data = {
    'Model': ['Baseline (Macro Only)', 'Our Model (tone_mkt)', 'Google Translated', 'Basic NLP'],
    'R-squared': [model_baseline.rsquared, model_mkt.rsquared, model_google.rsquared, model_basic.rsquared],
    'Tone_Coef': [None, model_mkt.params['tone_mkt'], model_google.params['tone_google'], model_basic.params['tone_basic']],
    'P-value': [None, model_mkt.pvalues['tone_mkt'], model_google.pvalues['tone_google'], model_basic.pvalues['tone_basic']]
}

summary_df = pd.DataFrame(summary_data)
print(summary_df)

In [ ]:
# 1. 실제 금리변동 vs 모델 예측치 비교 그래프
df_final['predicted_delta'] = model_robust.predict(X)

plt.figure(figsize=(12, 6))
plt.plot(df_final['Date'], df_final['delta_bok'], label='Actual (실제 금리변동)', color='black', alpha=0.5)
plt.plot(df_final['Date'], df_final['predicted_delta'], label='Predicted (모델 예측치)', color='red', linestyle='--')
plt.title('금리 변동 예측 모델 성능 확인 (Actual vs Predicted)')
plt.legend()
plt.show()

# 2. 분석 결과 파일로 내보내기
df_final.to_csv('final_analysis_result.csv', index=False)
print("최종 분석 결과가 저장되었습니다!")